In [25]:
# Run this to use from colab environment
#!pip install -q --upgrade git+https://github.com/naomiverkerk/TM10007.git


In [49]:
## Import
from sklearn import model_selection
from sklearn import metrics
from sklearn import feature_selection 
from sklearn import preprocessing
from sklearn import neighbors
from sklearn import svm
from sklearn import decomposition
from load_data import load_data
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [59]:
##Loading Data
data = load_data() 
X = data
X = X.replace(np.inf, np.nan)
Y = data['label']
del X['label']

In [103]:
## Split
X_train, X_test, Y_train, Y_test= train_test_split(X, Y, test_size = 0.2, random_state = 4, stratify = Y)

34


In [104]:

## Features weghalen met teveel missing values
acceptabele_ratio = 0.5
train_size = len(X_train.index)
removal_rate = round(train_size*acceptabele_ratio)

X_train = X_train.dropna(axis=1, thresh=removal_rate)
common_cols = list(set(X_train.columns).intersection(X_test.columns))
X_test = X_test[common_cols]

In [105]:
## Imputation met median
X_train_missing_median = X_train.fillna(X_train.median())
X_train_missing_median = np.nan_to_num(X_train_missing_median)
X_test_missing_median = X_test.fillna(X_test.median())
X_test_missing_median = np.nan_to_num(X_test_missing_median)

In [106]:
## Scaling
scaler = preprocessing.RobustScaler()
scaler.fit(X_train_missing_median)
X_train_scaled = scaler.transform(X_train_missing_median)
X_train_scaled = np.nan_to_num(X_train_scaled)
X_test_scaled = scaler.transform(X_test_missing_median)
X_test_scaled = np.nan_to_num(X_test_scaled)

In [107]:
## PCA
pca = decomposition.PCA(n_components=0.99, svd_solver= 'full')
pca.fit(X_train_scaled)
X_train_final = pca.transform(X_train_scaled)
X_test_final = pca.transform(X_test_scaled)
explained_variance = np.cumsum(pca.explained_variance_ratio_)


In [122]:
## KNN
cv_5fold= model_selection.StratifiedKFold(n_splits=5)
results= []
best_n_neighbors = []
k_list = list(range(1, 26, 2))
train_scores = []
validation_scores = []
all_train = []
all_validation = []

for train_index, validation_index in cv_5fold.split(X_train_final, Y_train):
    # Split the data properly
    X_validation = X_train_final[validation_index]
    Y_validation = Y_train[validation_index]

    X_train = X_train_final[train_index]
    Y_train = Y_train[train_index]

   for k in k_list:
        clf_knn = neighbors.KNeighborsClassifier(n_neighbors=k)
        clf_knn.fit(X_train, Y_train)

        # Test the classifier on the training data and plot
        train_proba = clf_knn.predict_proba(X_train)[:, 1]
        validation_proba = clf_knn.predict_proba(X_validation)[:, 1]
        
        score_train = metrics.roc_auc_score(Y_train, train_proba)
        score_validation = metrics.roc_auc_score(Y_validation, validation_proba)
        

        train_scores.append(score_train)
        validation_scores.append(score_validation)
        
    all_train.append(train_scores)
    all_validation.append(validation_scores)




#def KNN():
    

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 19)